# **OCR**

En este notebook veremos cómo **convertir diversos formatos de archivo a PDF** (por ejemplo, TIFF, ZIP con múltiples archivos, documentos de LibreOffice, etc.) y luego **parsearlos con [LlamaParse](https://pypi.org/project/llama-parse/)** para extraer texto de forma estructurada y confiable.  

El flujo principal será:

1. **Instalación de dependencias.**  
2. **Definición de la librería de funciones para conversión a PDF.**  
3. **Configuración previa (variables de entorno, etc.).**  
4. **Definir la ruta del documento PDF final.**  
5. **Establecer instrucciones de parseo (OCR y formato).**  
6. **Procesar el PDF con LlamaParse.**  
7. **Almacenar el texto resultante en un archivo local.**  
8. **Visualizar una sección del documento parseado.**  



## **Notas**
- Si necesitas extraer texto de un archivo que NO sea PDF, primero utiliza la **librería de conversión** (en la Celda 2) para transformarlo en un PDF válido.
- Configura la variable de entorno `LLAMA_CLOUD_API_KEY` con tu clave real para poder usar **LlamaParse**.
- Ajusta los parámetros de `LlamaParse` (por ejemplo, `premium_mode`, `language`, etc.) en función de tus necesidades de OCR y parseo.
- Para utilizar `premium_mode` consulta con el encargado de proporcionar las `API_KEY` debido a que su uso implica un costo adicional.[link text](https://)



#  1. Instalación de dependencias


In [18]:

!pip install llama-parse
!pip install PyPDF2
!pip install Pillow  # Para manejo de imágenes (TIFF) con PIL
!apt-get update
!apt-get install -y libreoffice


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,610 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,230 kB]
Fetched 3,969 kB in 2s (1,795 kB/s)
Reading package lists... Done
W: Skipping acquire of configured fi

In [3]:
import os
import zipfile
import subprocess
import shutil
from PIL import Image
from PyPDF2 import PdfReader, PdfWriter
from llama_parse import LlamaParse

# 2. Librería de Funciones para Conversión a PDF


In [4]:
def convert_file_to_pdf(path_original_file):
    """
    Convierte un archivo a PDF, manejando:
      - PDF: Retorna la misma ruta.
      - TIFF/TIF: Convierte usando PIL (multi-página).
      - ZIP: Descomprime, convierte cada archivo interno a PDF y unifica en uno solo.
      - Otros formatos (DOC, DOCX, etc.): Convierte usando LibreOffice.
    """
    # 1) Si ya es PDF, retornar la misma ruta
    if path_original_file.lower().endswith('.pdf'):
        print(f"El archivo '{path_original_file}' ya está en PDF.")
        return path_original_file

    # 2) Detectar extensión
    extension = os.path.splitext(path_original_file)[1].lower()

    # 2A) ZIP -> Descomprimir + convertir + unificar
    if extension == '.zip':
        return extract_and_unify_zip_to_pdf(path_original_file)

    # 2B) TIFF -> PIL multi-página
    if extension in ('.tif', '.tiff'):
        return convert_tiff_multi_page_to_pdf(path_original_file)

    # 2C) Otros formatos -> LibreOffice
    return convert_with_libreoffice(path_original_file)


def extract_and_unify_zip_to_pdf(zip_path):
    """
    Descomprime un .zip en una carpeta temporal, convierte cada archivo interno a PDF
    y unifica todos los PDFs resultantes en uno solo (orden alfabético).
    """
    # 1) Crear carpeta de extracción
    extract_dir = os.path.splitext(zip_path)[0]
    os.makedirs(extract_dir, exist_ok=True)

    # 2) Descomprimir
    try:
        with zipfile.ZipFile(zip_path, 'r') as zf:
            zf.extractall(extract_dir)
        print(f"Archivo ZIP extraído en: {extract_dir}")
    except Exception as e:
        print(f"[Error] Al descomprimir '{zip_path}': {e}")
        return None

    # 3) (Opcional) Eliminar el ZIP original
    try:
        os.remove(zip_path)
        print(f"Archivo ZIP original eliminado: {zip_path}")
    except Exception as e:
        print(f"[Advertencia] No se pudo eliminar el archivo ZIP original: {e}")

    # 4) Convertir cada archivo extraído a PDF
    pdf_paths = []
    for root, dirs, files in os.walk(extract_dir):
        for filename in files:
            extracted_path = os.path.join(root, filename)
            result_pdf = convert_file_to_pdf(extracted_path)
            if result_pdf:
                if isinstance(result_pdf, str):
                    pdf_paths.append(result_pdf)
                elif isinstance(result_pdf, list):
                    pdf_paths.extend(result_pdf)

    if not pdf_paths:
        print(f"[Aviso] No se encontraron archivos convertibles en '{zip_path}'.")
        return None

    # 5) Unificar todos los PDFs en uno solo
    merged_pdf_path = os.path.join(extract_dir, "merged_result.pdf")
    if not merge_pdfs(pdf_paths, merged_pdf_path):
        return None

    print(f"PDF unificado creado: {merged_pdf_path}")

    # 6) (Opcional) Eliminar los PDFs individuales
    for pdf_file in pdf_paths:
        if os.path.exists(pdf_file):
            os.remove(pdf_file)

    return merged_pdf_path


def merge_pdfs(pdf_paths, output_path):
    """
    Une una lista de rutas de PDF en un solo archivo.
    Retorna True si es exitoso, False si ocurre un error.
    """
    pdf_paths_sorted = sorted(pdf_paths)
    writer = PdfWriter()
    try:
        for pdf_file in pdf_paths_sorted:
            reader = PdfReader(pdf_file)
            for page in reader.pages:
                writer.add_page(page)

        with open(output_path, "wb") as out_f:
            writer.write(out_f)
        return True
    except Exception as e:
        print(f"[Error] Al unificar PDFs: {e}")
        return False


def convert_tiff_multi_page_to_pdf(path_tiff_file):
    """
    Convierte un archivo TIFF/TIF (posiblemente multi-página) a un solo PDF.
    """
    directory = os.path.dirname(path_tiff_file)
    base_name = os.path.splitext(os.path.basename(path_tiff_file))[0]
    pdf_name = base_name + ".pdf"
    pdf_path = os.path.join(directory, pdf_name)
    os.makedirs(directory, exist_ok=True)

    images = []
    try:
        with Image.open(path_tiff_file) as img:
            while True:
                frame = img.copy()
                if frame.mode in ("RGBA", "P"):
                    frame = frame.convert("RGB")
                images.append(frame)
                try:
                    img.seek(img.tell() + 1)
                except EOFError:
                    break
    except Exception as e:
        print(f"[Error] Al abrir {path_tiff_file}: {e}")
        return None

    if not images:
        print(f"[Aviso] No se pudieron extraer páginas de {path_tiff_file}")
        return None

    try:
        images[0].save(
            pdf_path,
            "PDF",
            resolution=100.0,
            save_all=True,
            append_images=images[1:]
        )
        print(f"PDF multi-página creado: {pdf_path}")
        os.remove(path_tiff_file)  # Eliminar original
        return pdf_path
    except Exception as e:
        print(f"[Error] Al crear el PDF a partir de TIFF: {e}")
        return None


def convert_with_libreoffice(path_original_file):
    """
    Convierte archivos de texto (DOC, DOCX, ODT, etc.) a PDF usando LibreOffice.
    Ajustar la ruta del comando 'soffice' según tu sistema operativo.
    """
    directory = os.path.dirname(path_original_file)
    base_name = os.path.splitext(os.path.basename(path_original_file))[0]
    pdf_path = os.path.join(directory, base_name + ".pdf")
    os.makedirs(directory, exist_ok=True)

    try:
        # Ajusta la ruta de soffice si usas Linux u otra plataforma.
        subprocess.run(
            [
                "/Applications/LibreOffice.app/Contents/MacOS/soffice",
                "--headless",
                "--convert-to", "pdf",
                path_original_file,
                "--outdir", directory
            ],
            check=True
        )
        print(f"PDF creado a partir de '{path_original_file}': {pdf_path}")
        os.remove(path_original_file)  # Eliminar original
        return pdf_path
    except Exception as e:
        print(f"[Error] Al convertir '{path_original_file}' a PDF: {e}")
        return None


# 3. Configuración Previa


In [5]:
import nest_asyncio
nest_asyncio.apply()

# Ajusta tu clave de LlamaParse aquí:
os.environ["LLAMA_CLOUD_API_KEY"] = "llx-"


# 4. Definir la ruta del documento PDF

Puedes encontrar algunos archivos de ejemplo en el siguiente [link](https://drive.google.com/drive/folders/1KKJqJ_Z-H2JP_NpA3zHFPU7839--QzF6?usp=sharing). Luego, este archivo debe ser cargado en /content/


In [10]:
original_path = "/content/29701.tif"
convert_file_to_pdf(original_path)

PDF multi-página creado: /content/29701.pdf


'/content/29701.pdf'

In [11]:
path_pdf_document = "/content/29701.pdf"
print(f"Usaremos el archivo PDF en: {path_pdf_document}")

Usaremos el archivo PDF en: /content/29701.pdf


# 5. Definición de las Instrucciones de Parseo

Consiste en explicar, en términos generales la estructura del documento que se proporcionará y lo que se espera que haga el parser (por ejemplo, recuperar integramente el contenido y en idioa español)


In [12]:
parsing_instructions = '''
The document contains structured text that includes headers, dates, names of individuals, institutions, and places, as well as numbered sections, lists, and tables. Many of these documents feature text that is highly deteriorated and requires careful interpretation, contextualization, and reconstruction. Non-essential elements, such as scratches, signatures, and diagonal annotations, must be omitted.
To ensure the integrity of the extracted information, the text must be preserved in its literal form without summarizing, paraphrasing, or modifying its meaning. Treat each document as evidentiary material, prioritizing rigorous and faithful extraction practices.
Recognize that most documents are typewritten, which introduces specific challenges such as ink smudges, duplicated letters, and words split across lines. Proactively correct these errors, ensuring clarity and precision in the recovered content.
While processing, handle page breaks to maintain the narrative flow, and retain the original structure of tables and lists without flattening their format. Extract and clearly highlight key names, dates, and places. Quotes, dialogues, abbreviations, and codes must be preserved exactly as they appear. Non-textual elements should be omitted to focus solely on the written content.
The output should adhere to Markdown formatting conventions but must not include code block tags such as markdown. Use bold formatting for headers, numbered or bulleted lists for structured sections, block quotes for quoted material, and Markdown-compatible tables for tabular data. Narrative text should be continuous, providing both factual information and detailed descriptions, while preserving the integrity and original context of the document. Additionally, prioritize correcting OCR-related errors caused by typewriter artifacts whenever possible.
The response must be exclusively in the original language of the document, which is generally Spanish. No translation or language modification is allowed.
'''

print("Instrucciones de parseo definidas.")


Instrucciones de parseo definidas.


# 6. Carga y Procesamiento del Documento con LlamaParse


In [13]:
# Cargamos el documento usando LlamaParse con las instrucciones definidas
document = LlamaParse(
    result_type="markdown",
    premium_mode=False,
    parsing_instructions=parsing_instructions,
    language='es',
    skip_diagonal_text=True,
    do_not_unroll_columns=False
).load_data(path_pdf_document)

print("Documento parseado correctamente.")
print(f"Total de secciones extraídas: {len(document)}")


Started parsing the file under job_id 42e95bfe-0aff-4223-88dd-17d3eaaa1189
.Documento parseado correctamente.
Total de secciones extraídas: 13


# 7. Guardar el Texto Recuperado en un Archivo


In [14]:

full_text = ''
for i in range(len(document)):
    full_text += '\n\n'
    full_text += document[i].text

# Crear carpeta y archivo de salida
directory = "/content/outputs"
file_name = "02.txt"
os.makedirs(directory, exist_ok=True)
file_path = os.path.join(directory, file_name)

# Escribir el contenido parseado en un .txt
with open(file_path, "w") as file:
    file.write(full_text)

print(f"El archivo parseado ha sido guardado en: {file_path}")


El archivo parseado ha sido guardado en: /content/outputs/02.txt


# 8. Visualización de una Página o Sección


In [17]:
# Puedes cambiar el índice para ver otras partes del documento

pagina_a_visualizar = 3

if pagina_a_visualizar < len(document):
    print(document[pagina_a_visualizar].text)
else:
    print(f"Índice fuera de rango. El documento sólo tiene {len(document)} partes extraídas.")


# Ministerio de Energia

# SSC

# SuPENNTENDEA(U DeelecircIDAD

# Gobierno de Chile

Regularizar y solucionar problemas formales de las CUOs que fueran necesarios para permitir el desarrollo de los proyectos amparados en ellas.

Teniendo en cuenta la preocupación que ha puesto el Estado en el tema descrito; Chungungo tomó la decisión de desarrollar el Proyecto en un terreno de propiedad fiscal sobre el cual el MBN otorgó una CUO que originalmente fue otorgada a la empresa Hydrochile SA. (en adelante "Hydrochile") por medio de la Resolución Exenta N° 758 del julio de 2015, del MBN, CUO que actualmente es de propiedad de Chungungo.

En forma previa y luego paralela a las gestiones de obtención del ICC y la RCA del Proyecto; a mediados del año 2017 Chungungo inició negociaciones con Hydrochile para adquirir la CUO, previo a lo cual debían solucionarse problemas formales con el MBN, ya que luego de la obtención de la CUO Hydrochile dejó pendiente la firma del respectivo contrato de concesi